In [53]:
from lib import get_prompts
import openai
import re
import time

import numpy as np
import random

from tqdm import tqdm
from datasets import load_dataset
import re


In [5]:
dataset = load_dataset("hotpot_qa", "fullwiki", trust_remote_code=True)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 90447
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
})

In [25]:
test_data = dataset["validation"]


In [42]:
def construct_multi_shot(num_shot = 4):
    instruction = 'Answer the final question following the exact format of the given examples. Do not output anything else.\n\nExample problems:\n\n'
    example_demonstrations = get_prompts(num_shot)
    prompt = instruction
    for line in example_demonstrations:
        s = line
        prompt += s + '\n'
    prompt += '\n'
    prompt += f"Question to answer:\n\nQ:"
    return prompt

def construct_0_shot():
  instruction = 'Answer the question, with your final answer as: Therefore the answer is: <final answer>. Think step by step.'
  return instruction


In [62]:
def get_question(prompt, index):
    question = test_data['question'][index]
    prompt_q = prompt + '\n' + question
    return prompt_q

# print(get_question(construct_0_shot()))

In [57]:
# test

def query(prompt):
    client = openai.OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Follow the given instructions and answer the question."},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return completion.choices[0].message.content




In [60]:
def trim_non_alphabetical(s):
    return re.sub(r'^[^a-zA-Z]+|[^a-zA-Z]+$', '', s)

def extract_answer(text):
    match = re.search(r"is:(.*)", text)
    if match:
        return trim_non_alphabetical(match.group(1).strip())
    else:
        return None

In [64]:
random_indexes = random.sample(range(len(test_data)), 20)
answers = [test_data['answer'][i] for i in random_indexes]

answers

for index in tqdm(random_indexes):
    prompt = get_question(construct_multi_shot(), index)
    result = query(prompt)
    print(result)
    print(extract_answer(result))

  5%|▌         | 1/20 [00:01<00:19,  1.04s/it]A: The Argentinian retired defensive midfielder and current manager of C.D. Guadalajara is Ricardo La Volpe. Ricardo La Volpe was traded to Parma at the end of the 1999–2000 season. So the answer is: Ricardo La Volpe.
Ricardo La Volpe
 10%|█         | 2/20 [00:01<00:17,  1.03it/s]A: An American Werewolf in Paris was a partial sequel to the original film An American Werewolf in London, which starred David Naughton. Thus, the comedy film starring David Naughton is the one referred to. So the answer is: David Naughton.
David Naughton
 15%|█▌        | 3/20 [00:02<00:14,  1.17it/s]A: The man that holds a record with John Boulger is Michael van Gerwen, who won the European Championship twice, in 2014 and 2017. So the answer is: European Championship.
European Championship
 20%|██        | 4/20 [00:03<00:16,  1.01s/it]A: Patrol Squadron Four (VP-4) is based at the Naval Station known formerly as NAS Barbers Point. So the answer is: NAS Barbers Poi

In [65]:
answers

['Matías Jesús Almeyda',
 'David Naughton, Jenny Agutter and Griffin Dunne',
 "London Riders' Championship",
 'Marine Corps Air Station Kaneohe Bay',
 'Clara Petacci',
 'Justin Adler',
 'Robert Jenrick',
 'Liam Cunningham',
 'Tyler "Ty" Mendoza',
 '22 September 2015',
 'Hideyoshi',
 'May 22',
 'Cartoon Network Studios',
 'University of California, Berkeley',
 'Microsoft Office',
 "India's first dedicated multi-wavelength space observatory.",
 'Marvel Comics',
 'no',
 'Swansea City',
 'cocktails']

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5134a3cb-273d-4bd5-9fec-9f3444f1bc49' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>